In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory



# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import sklearn
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score

In [3]:
df = pd.read_csv('Offensive/train_data.csv')

In [4]:
df

,comments,tagging
0,Get fucking real dude.,1
1,She is as dirty as they come and that crook ...,1
2,why did you fuck it up. I could do it all day...,1
3,Dude they dont finish enclosing the fucking s...,1
4,WTF are you talking about Men? No men thats n...,1
...,...,...
19996,I dont. But what is complaining about it goi...,0
19997,Bahah yeah i&;m totally just gonna&; get pis...,0
19998,hahahahaha >:) im evil mwahahahahahahahahaha,0
19999,What&;s something unique about Ohio? :),0


In [5]:
df1 = pd.read_csv("Offensive/train.csv")


In [6]:
df1

,id,label,tweet
0,1,0,@user when a father is dysfunctional and is s...
1,2,0,@user @user thanks for #lyft credit i can't us...
2,3,0,bihday your majesty
3,4,0,#model i love u take with u all the time in ...
4,5,0,factsguide: society now #motivation
...,...,...,...
31957,31958,0,ate @user isz that youuu?ðððððð...
31958,31959,0,to see nina turner on the airwaves trying to...
31959,31960,0,listening to sad songs on a monday morning otw...
31960,31961,1,"@user #sikh #temple vandalised in in #calgary,..."


In [7]:
df1 = df1[['tweet','label']].copy()

In [8]:
df1.rename(columns = {'tweet' : 'comments', 'label' : 'tagging'}, inplace = True)

In [9]:
df_Offensive = pd.read_csv("Offensive/labeled_data.csv")

In [10]:
df_Offensive

,Unnamed: 0,count,hate_speech,offensive_language,neither,class,tweet
0,0,3,0,0,3,2,!!! RT @mayasolovely: As a woman you shouldn't...
1,1,3,0,3,0,1,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...
2,2,3,0,3,0,1,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...
3,3,3,0,2,1,1,!!!!!!!!! RT @C_G_Anderson: @viva_based she lo...
4,4,6,0,6,0,1,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...
...,...,...,...,...,...,...,...
24778,25291,3,0,2,1,1,you's a muthaf***in lie &#8220;@LifeAsKing: @2...
24779,25292,3,0,1,2,2,"you've gone and broke the wrong heart baby, an..."
24780,25294,3,0,3,0,1,young buck wanna eat!!.. dat nigguh like I ain...
24781,25295,6,0,6,0,1,youu got wild bitches tellin you lies


In [11]:
df_Offensive = df_Offensive[['tweet','class']].copy()

In [12]:
df_Offensive.head()

,tweet,class
0,!!! RT @mayasolovely: As a woman you shouldn't...,2
1,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...,1
2,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...,1
3,!!!!!!!!! RT @C_G_Anderson: @viva_based she lo...,1
4,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...,1


In [13]:
print(sum(df_Offensive["class"]==0))
print(sum(df_Offensive["class"]==1))
print(sum(df_Offensive["class"]==2))

1430
19190
4163


In [14]:
#df_Offensive[df_Offensive["class"]==0]["class"] = 1
df_Offensive.loc[(df_Offensive['class'] == 0), 'class'] = 1

In [15]:
#df_Offensive[df_Offensive["class"]==2]["class"] = 0
df_Offensive.loc[(df_Offensive['class'] == 2), 'class'] = 0

In [16]:
print(sum(df_Offensive["class"]==0))
print(sum(df_Offensive["class"]==1))
print(sum(df_Offensive["class"]==2))

4163
20620
0


In [17]:
df_Offensive.rename(columns = {'tweet' : 'comments', 'class' : 'tagging'}, inplace = True)

In [18]:
df = pd.concat([df, df1], ignore_index=True)

In [19]:
df = pd.concat([df, df_Offensive], ignore_index=True)


In [20]:
df_Offensive

,comments,tagging
0,!!! RT @mayasolovely: As a woman you shouldn't...,0
1,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...,1
2,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...,1
3,!!!!!!!!! RT @C_G_Anderson: @viva_based she lo...,1
4,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...,1
...,...,...
24778,you's a muthaf***in lie &#8220;@LifeAsKing: @2...,1
24779,"you've gone and broke the wrong heart baby, an...",0
24780,young buck wanna eat!!.. dat nigguh like I ain...,1
24781,youu got wild bitches tellin you lies,1


In [21]:
df_Offensive = pd.DataFrame(df_Offensive, columns=['Name', 'Marks'])

In [22]:
df

,comments,tagging
0,Get fucking real dude.,1
1,She is as dirty as they come and that crook ...,1
2,why did you fuck it up. I could do it all day...,1
3,Dude they dont finish enclosing the fucking s...,1
4,WTF are you talking about Men? No men thats n...,1
...,...,...
76741,you's a muthaf***in lie &#8220;@LifeAsKing: @2...,1
76742,"you've gone and broke the wrong heart baby, an...",0
76743,young buck wanna eat!!.. dat nigguh like I ain...,1
76744,youu got wild bitches tellin you lies,1


In [23]:
df.isnull().values.any()

False

In [24]:
df. isnull(). sum()

comments    0
tagging     0
dtype: int64

In [25]:
import re

#set up punctuations we want to be replaced
REPLACE_NO_SPACE = re.compile("(\.)|(\;)|(\:)|(\!)|(\')|(\?)|(\,)|(\")|(\|)|(\()|(\))|(\[)|(\])|(\%)|(\$)|(\>)|(\<)|(\{)|(\})")
REPLACE_WITH_SPACE = re.compile("(<br\s/><br\s/?)|(-)|(/)|(:).")

In [26]:
import preprocessor as p

# custum function to clean the dataset (combining tweet_preprocessor and reguar expression)
def clean_tweets(df):
  tempArr = []
  for line in df:
    # send to tweet_processor
    tmpL = p.clean(line)
    # remove puctuation
    tmpL = REPLACE_NO_SPACE.sub("", tmpL.lower()) # convert all tweets to lower cases
    tmpL = REPLACE_WITH_SPACE.sub(" ", tmpL)
    tempArr.append(tmpL)
  return tempArr

In [27]:
train_tweet = clean_tweets(df["comments"])
train_tweet = pd.DataFrame(train_tweet)

In [28]:
df["comments"] = train_tweet

In [29]:
from sklearn.model_selection import train_test_split
X = df['comments']
y = df['tagging']

In [30]:
y

0        1
1        1
2        1
3        1
4        1
        ..
76741    1
76742    0
76743    1
76744    1
76745    0
Name: tagging, Length: 76746, dtype: int64

In [31]:
X

0                                    get fucking real dude
1        she is as dirty as they come and that crook re...
2        why did you fuck it up i could do it all day t...
3        dude they dont finish enclosing the fucking sh...
4        wtf are you talking about men no men thats not...
                               ...                        
76741    yous a muthaf***in lie & right his tl is trash...
76742    youve gone and broke the wrong heart baby and ...
76743    young buck wanna eat dat nigguh like i aint fu...
76744                youu got wild bitches tellin you lies
76745    ~~ruffled  ntac eileen dahlia   beautiful colo...
Name: comments, Length: 76746, dtype: object

In [32]:
train_X,val_X,train_y,val_y=(train_test_split(X, y, random_state = 0,test_size=0.20))
v_X=val_X

In [33]:
train_X

61017                     famous women be the biggest hoes
37509                                                 look
65083                 my nigguhme naturally forever be fly
62797    i really hate a fat bitch cause im bony love s...
17912                                  i hate that so much
                               ...                        
21243    so to be performing tonight for alongside with...
45891      away the century plant is presenting with buds 
42613                    apeezy   cloudy room listen below
43567                                              the gun
68268     cant no bitch tell me a nigga not mines until...
Name: comments, Length: 61396, dtype: object

In [34]:
#output = pd.DataFrame({'comments': v_X , 'tagging': predictions})
#output.to_csv('my_submission.csv', index=False)
#print("Your submission was successfully saved!")

In [35]:
phrase = train_X.head(1)
print(phrase)
vect=CountVectorizer()

61017    famous women be the biggest hoes
Name: comments, dtype: object


In [36]:
vect.fit(phrase)
print("vocabulary size: {}".format(len(vect.vocabulary_)))
print("vocabulary content:\n {}".format(vect.vocabulary_))

vocabulary size: 6
vocabulary content:
 {'famous': 2, 'women': 5, 'be': 0, 'the': 4, 'biggest': 1, 'hoes': 3}


In [37]:
bag_of_words=vect.transform(phrase)
print(bag_of_words)

  (0, 0)	1
  (0, 1)	1
  (0, 2)	1
  (0, 3)	1
  (0, 4)	1
  (0, 5)	1


In [38]:
print("bag_of_words as an array:\n{}".format(bag_of_words.toarray()))

bag_of_words as an array:
[[1 1 1 1 1 1]]


In [39]:
vect.get_feature_names()

c:\Users\bhupeshbhatt\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


['be', 'biggest', 'famous', 'hoes', 'the', 'women']

In [40]:
train_X.head()

61017                     famous women be the biggest hoes
37509                                                 look
65083                 my nigguhme naturally forever be fly
62797    i really hate a fat bitch cause im bony love s...
17912                                  i hate that so much
Name: comments, dtype: object

In [41]:
train_X.head()

61017                     famous women be the biggest hoes
37509                                                 look
65083                 my nigguhme naturally forever be fly
62797    i really hate a fat bitch cause im bony love s...
17912                                  i hate that so much
Name: comments, dtype: object

In [42]:
vectorizer = CountVectorizer()
train_X = vectorizer.fit_transform(train_X)
val_X = vectorizer.transform(val_X)

In [56]:
import pickle
pickle.dump(vectorizer, open("cv.pickel", "wb"))

In [44]:
feature_names = vectorizer.get_feature_names()

In [45]:
train_X.shape

(61396, 37070)

In [46]:
from sklearn.svm import SVC # "Support vector classifier"  
model = SVC(kernel='linear', random_state=0,C=10)
model.fit(train_X,train_y)
predictions = model.predict(val_X)

In [57]:

pickle.dump(model, open("lr.pickel", "wb"))

In [48]:
output = pd.DataFrame({'comments': v_X , 'tagging': predictions})
output.to_csv('my_submission.csv', index=False)

In [49]:
confusion = confusion_matrix(val_y,predictions)
print(confusion)

[[8298  944]
 [ 632 5476]]


In [50]:
from sklearn.metrics import accuracy_score
print(accuracy_score(val_y,predictions))

0.897328990228013


In [51]:
import matplotlib.pyplot as plt
%matplotlib inline  

In [52]:
def plot_coefficients(classifier, feature_names, top_features=20):
 coef = classifier.coef_
 coef = coef.toarray()
 out = np.argsort(coef)
 coef = coef.transpose()
 out = out.transpose()[-100:]
 out = out[::-1]
 print(out.shape)
 out = out.transpose()
 #print(coef[out])
 feature_names = np.array(feature_names)
 print(feature_names[out])

In [53]:
plot_coefficients(model,feature_names)

(100, 1)
[['bitchyall' 'casey' 'pussys' 'niggers' 'faggot' 'knob' 'intend'
  'emopoor' 'fifteen' 'nigglet' 'hash' 'stewart' 'bitchsee' 'nigguh'
  'hoes' 'rednecks' 'bitches' 'pussies' 'freakn' 'travelled' 'livesbut'
  'barnicle' 'niggah' 'niccas' 'nigs' 'specifically' 'rinds' 'extremist'
  'seventies' 'scurvy' 'faggots' 'reruns' 'dependent' 'nigger'
  'sympathize' 'pussy' 'hoe' 'bitch' 'accusations' 'literary' 'ampe'
  'reali' 'gingers' 'bsa' 'debacle' 'petous' 'dickbwaha' 'expes'
  'intoxication' 'nig' 'dictionaries' 'amrica' 'coincidence' 'cunts'
  'seller' 'omarosas' 'facilitate' 'documented' 'emos' 'fag' 'sturgeon'
  'pos' 'censored' 'existence' 'binds' 'sayalso' 'constructive' 'festive'
  'rengel' 'niglet' 'pecan' 'cboys' 'gradeschool' 'humiliated'
  'strangely' 'caint' 'mf' 'mcnab' 'install' 'leakage' 'itas' 'rts'
  'fack' 'pizzas' 'freplce' 'queers' 'critiquing' 'twat' 'dyke' 'ohmygod'
  'retardedn' 'ussy' 'bitcheshard' 'mayag' 'plaza' 'animalpussy' 'sweeet'
  'niggar' 'whites' 